In [2]:
import json
import logging
import math
import random
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import List, Tuple

import pandas
import pyarrow.parquet as pq
import sentence_transformers.losses as losses
from datasets import Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    models,
)
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers.readers import InputExample
from transformers import (
    AutoTokenizer,
    TrainerCallback,
    TrainerControl,
    TrainerState,
    BitsAndBytesConfig,
    AutoModelForMaskedLM,
    AutoModel,
    QuantoConfig,
)
from pprint import pprint
import torch
from quanto import quantize, freeze, qint8, qfloat8

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO)

In [4]:
def make_sentence_transformer(
    model_name: str, max_seq_length: int = 512
) -> SentenceTransformer:
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    # tokenizer.model_max_length = max_seq_length  # Set the max length for the model
    # tokenizer.padding_side = (
    #     "right"  # You can set "left" if you want to pad on the left side
    # )
    # # tokenizer.pad_token = tokenizer.eos_token  # Ensure the pad token is set
    # model = SentenceTransformer(model_name)
    # # Add the padding and truncation to the encode method
    # model.tokenizer = tokenizer
    # model.tokenizer_kwargs = {
    #     "padding": "max_length",
    #     "truncation": True,
    #     "max_length": max_seq_length,
    #     "return_tensors": "pt",  # Assuming you want PyTorch tensors as output
    # }
    # return model


    # Load model directly
    # quantization_config = BitsAndBytesConfig(load_in_8bit = True)
    # quantization_config = QuantoConfig(weights="int8")

    # word_embedding_model = AutoModel.from_pretrained(model_name, quantization_config=quantization_config)

    # # Apply mean pooling to get one fixed sized sentence vector
    # pooling_model = models.Pooling(max_seq_length,
    #                             pooling_mode_cls_token=False,
    #                             pooling_mode_max_tokens=False,
    #                             pooling_mode_mean_tokens=True)

    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    # tokenizer.model_max_length = max_seq_length  # Set the max length for the model
    # tokenizer.padding_side = (
    #     "right"  # You can set "left" if you want to pad on the left side
    # )
    # # tokenizer.pad_token = tokenizer.eos_token  # Ensure the pad token is set
    # model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    # # Add the padding and truncation to the encode method
    # model.tokenizer = tokenizer
    # model.tokenizer_kwargs = {
    #     "padding": "max_length",
    #     "truncation": True,
    #     "max_length": max_seq_length,
    #     "return_tensors": "pt",  # Assuming you want PyTorch tensors as output
    # }
    # return model

    word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False,
        pooling_mode_mean_tokens=True,
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    # quantize(model, weights=qfloat8, activations=qfloat8)
    # freeze(model)
    return model

In [5]:
jerteh = make_sentence_transformer("jerteh/Jerteh-355")

Some weights of RobertaModel were not initialized from the model checkpoint at jerteh/Jerteh-355 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu


In [8]:
jerteh.half()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)